In [1]:
import copy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("../data/samsung_health_sleep.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.drop(["total_sleep_time_weight", "factor_01", "factor_02", "factor_03", "factor_04", "factor_05", "factor_06", "factor_07", "factor_08", "factor_09", "factor_10", "latency_weight", "latency_score", "deep_weight", "rem_weight", "wake_weight"], axis=1)

Regression with NN

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error

features = [
    "mental_recovery",
	"wake_score",
    "deep_score",
	"sleep_efficiency_with_latency",
	"total_rem_duration",
	"physical_recovery",
	"movement_awakening",
	"rem_score",
	"sleep_cycle",
	"total_light_duration",
	"efficiency",
	"sleep_duration" 
]

X = df[features].values
y = df["sleep_score"].values.reshape(-1,1)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

x_train = scaler_X.fit_transform(x_train)
x_test = scaler_X.transform(x_test)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [5]:
class SleepRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

In [6]:
X_train_t = torch.tensor(x_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test_t = torch.tensor(x_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [7]:
model = SleepRegressor(input_dim=x_train.shape[1])

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 500
for epoch in range(epochs):
    model.train()

    y_pred = model(X_train_t)
    loss = criterion(y_pred, y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")


Epoch 0 | Loss: 1.1002
Epoch 50 | Loss: 0.0702
Epoch 100 | Loss: 0.0332
Epoch 150 | Loss: 0.0235
Epoch 200 | Loss: 0.0177
Epoch 250 | Loss: 0.0131
Epoch 300 | Loss: 0.0095
Epoch 350 | Loss: 0.0067
Epoch 400 | Loss: 0.0046
Epoch 450 | Loss: 0.0030


In [8]:
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_t)

y_pred_test = scaler_y.inverse_transform(y_pred_test.numpy())
y_test_real = scaler_y.inverse_transform(y_test)

rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_test))
print(f"RMSE: {rmse:.2f}")


RMSE: 4.36
